# Notebook Regarding Discussion of $\bar{V_R^2}$

## Loading Data

In [ ]:

import sys
sys.path.append("../gaia_tools/")
sys.path.append("../scripts/")
import data_analysis
import covariance_generation as cov
from import_functions import *
from data_plot import sample_distribution_galactic_coords, plot_radial_distribution, plot_distribution, display_polar_histogram, plot_variance_distribution, plot_velocity_distribution
import numpy as np
import emcee
from functools import reduce
import time, timeit
import transformation_constants
import datetime as dt
import photometric_cut
import os
import pickle
from pathlib import Path
import argparse
import random
import matplotlib.pyplot as plt


In [ ]:
galcen_data = import_baseline_sample()

In [ ]:
# Generate bins
bin_collection = data_analysis.get_collapsed_bins(data = galcen_data,
                                                      theta = (0, 1),
                                                      BL_r_min = min_r - 1,
                                                      BL_r_max = max_r + 1,
                                                      BL_z_min = -200,
                                                      BL_z_max = 200,
                                                      N_bins = (50, 1),
                                                      r_drift = False,
                                                      debug = False)

In [ ]:
bin_mids = []
for i, bin in enumerate(bin_collection.bins):
    bin_mids.append(np.median(bin.r_boundaries))

## XX Component Check

In [ ]:
new_xx = []
for i, bin in enumerate(bin_collection.bins):
    nom = np.var(bin.data.v_phi)
    denom = np.mean(bin.data.v_r)**2 + np.var(bin.data.v_r)
    new_xx.append(nom/denom)

plt.plot(np.array(bin_mids)/1000, new_xx)
plt.xlabel(r'$r$ [kpc]', fontdict={'fontsize' : 16})
plt.ylabel(r'$X^2$',  fontdict={'fontsize' : 16})
plt.xticks(np.linspace(5,12,6))

## Checking components in $\overline{v^2_R} = \overline{v_R}^2 + \sigma^2_R$

In [ ]:
# Compute the variance and mean of v_r in bins

velocity_var = []
for i, bin in enumerate(bin_collection.bins):
    var_i = np.var(bin.data['v_r'])
    velocity_var.append(var_i)
velocity_var = np.array(velocity_var)

mean_vel = []
for i, bin in enumerate(bin_collection.bins):
    mean_vel.append(np.mean(bin.data.v_r))
mean_vel = np.array(mean_vel)

### Checking $\sigma_R^2$ component

In [ ]:
plt.plot(np.array(bin_mids)/1000, np.sqrt(velocity_var))
plt.xlabel(r'$r$ [kpc]', fontdict={'fontsize' : 16})
plt.ylabel(r'$\sqrt{\sigma_r^2}$',  fontdict={'fontsize' : 16})
plt.xticks(np.linspace(5,12,6))

In [ ]:
plt.plot(np.array(bin_mids)/1000, velocity_var)
plt.xlabel(r'$r$ [kpc]', fontdict={'fontsize' : 16})
plt.ylabel(r'$\sigma_r^2$',  fontdict={'fontsize' : 16})
plt.xticks(np.linspace(5,12,6))

### Checking $\bar{v_R}^2$ component

In [ ]:
mean_sqrared = mean_vel**2

plt.scatter(np.array(bin_mids)/1000, mean_sqrared, s=10, color='black', label=r'$\bar{v_r}^2$')
plt.scatter(np.array(bin_mids)/1000, velocity_var, s=10, color='red', label=r'$\sigma^2_r$')

overall_mean = np.mean(mean_sqrared[~np.isnan(mean_sqrared)])

plt.hlines(overall_mean, 
                    bin_mids[0]/1e3, bin_mids[-1]/1e3, 
                    linestyles='--', 
                    label="Mean " + r"$\overline{v_r}^2$:" + "{}".format(np.round(overall_mean)))
plt.xlabel(r'$r$ [kpc]', fontdict={'fontsize' : 16})
plt.ylabel(r'$\overline{v_r}^2$',  fontdict={'fontsize' : 16})
plt.xticks(np.linspace(5,12,6))
plt.legend()


In [ ]:
plt.scatter(np.array(bin_mids)/1000, mean_sqrared, s=4, color='black')

overall_mean = np.mean(mean_sqrared[~np.isnan(mean_sqrared)])

plt.hlines(overall_mean, 
                    bin_mids[0]/1e3, bin_mids[-1]/1e3, 
                    linestyles='--', 
                    label="Mean: {}".format(np.round(overall_mean)))
plt.xlabel(r'$r$ [kpc]', fontdict={'fontsize' : 16})
plt.ylabel(r'$\overline{v_r}^2$ [km/s]',  fontdict={'fontsize' : 16})
plt.xticks(np.linspace(5,12,6))
plt.legend()
overall_mean

In [ ]:
plt.scatter(np.array(bin_mids)/1000, mean_vel, s=4, color='black')

overall_mean = np.mean(mean_vel[~np.isnan(mean_vel)])

plt.hlines(overall_mean, 
                    bin_mids[0]/1e3, bin_mids[-1]/1e3, 
                    linestyles='--', 
                    label="Mean: {}".format(np.round(overall_mean,2)))
plt.xlabel(r'$r$ [kpc]', fontdict={'fontsize' : 16})
plt.ylabel(r'$\overline{v_r}$ [km/s]',  fontdict={'fontsize' : 16})
plt.xticks(np.linspace(5,12,6))
plt.legend()


### Checking $\overline{v_R}^2 + \sigma^2_R$

In [ ]:
mean_sqrared = mean_vel**2

plt.scatter(np.array(bin_mids)/1000, mean_sqrared+velocity_var, s=10, color='black', label=r'$\overline{v_R}^2 + \sigma^2_R$')
plt.scatter(np.array(bin_mids)/1000, velocity_var, s=10, color='red', label=r'$\sigma^2_r$')

#overall_mean = np.mean(mean_sqrared[~np.isnan(mean_sqrared)])

plt.xlabel(r'$r$ [kpc]', fontdict={'fontsize' : 16})
plt.ylabel(r'$\overline{v_R}^2 + \sigma^2_R$',  fontdict={'fontsize' : 16})
plt.xticks(np.linspace(5,12,6))
plt.legend()


## Astropy Sanity Check

In [ ]:
import astropy.coordinates as coord
import astropy.units as u

icrs_star = icrs_data.iloc[0]

c1 = coord.SkyCoord(ra=icrs_star.ra*u.degree, 
                    dec=icrs_star.dec*u.degree,
                    distance=icrs_star.r_est*u.pc,
                    pm_ra_cosdec=icrs_star.pmra*u.mas/u.yr,
                    pm_dec=icrs_star.pmdec*u.mas/u.yr,
                    radial_velocity=icrs_star.radial_velocity*u.km/u.s,
                    frame='icrs')

In [ ]:
v_sun = [-11.1, 232.24, 7.25]* (u.km / u.s)

gc_frame = coord.Galactocentric(galcen_distance=8.178*u.kpc,
    galcen_v_sun=v_sun,
    z_sun=17*u.pc)

gc1 = c1.transform_to(gc_frame)
gc1

In [ ]:
coord.Galactocentric()

In [ ]:
print(galcen_data.iloc[0])

----